In [1]:
import re
import pandas as pd
import joblib
df=pd.read_csv("../Data/balanced_f1_dataset.csv")
df.head()
df.info()
df.isna().sum()
df['winner_name'].value_counts()

def clean_text(text):
    text=text.lower()
    text=re.sub(r"\s+"," ",text)
    text=re.sub(r"\[[0-9]+\]","",text)
    return text.strip()

df["clean_text"]=df["full_text"].apply(clean_text)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   race_id      45 non-null     int64 
 1   race_name    45 non-null     object
 2   race_date    45 non-null     object
 3   race_year    45 non-null     int64 
 4   winner_name  45 non-null     object
 5   winner_team  35 non-null     object
 6   full_text    45 non-null     object
 7   label        45 non-null     int64 
dtypes: int64(3), object(5)
memory usage: 2.9+ KB


In [2]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["label"]=le.fit_transform(df["winner_name"])
joblib.dump(le, "models/label_encoder.pkl")
print(list(le.classes_))

['Charles Leclerc', 'George Russell', 'Lando Norris', 'Lewis Hamilton', 'Max Verstappen', 'Oscar Piastri', 'Pierre Gasly', 'Sergio Pérez', 'Valtteri Bottas']


In [3]:
print(df["winner_name"].value_counts())

winner_name
Pierre Gasly       5
Charles Leclerc    5
Valtteri Bottas    5
Lewis Hamilton     5
Lando Norris       5
Max Verstappen     5
Sergio Pérez       5
George Russell     5
Oscar Piastri      5
Name: count, dtype: int64


In [4]:
counts = df['label'].value_counts()
rare = counts[counts == 1].index
rare


Index([], dtype='int64', name='label')

In [5]:
for r in rare:
    df = pd.concat([df, df[df['label'] == r]])


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]     
)


In [7]:
import os 
os.makedirs("Data/Processed",exist_ok=True)
train_df=pd.DataFrame({"text": X_train, "label": y_train})
text_df=pd.DataFrame({"text": X_test, "label": y_test})

train_df.to_csv("Data/Processed/train.csv",index=False)
text_df.to_csv("Data/Processed/test.csv",index=False)   
print("Preprocessed data saved to Data/Processed/train.csv and Data/Processed/test.csv")

Preprocessed data saved to Data/Processed/train.csv and Data/Processed/test.csv
